In [ ]:
# Python imports
import logging
import xarray as xr
import pandas as pd

# Libs
import geopandas as gpd
import shapely.geometry as shpg

# Locals
import oggm.cfg as cfg
from oggm import utils, workflow, tasks

# For timing the run
import time
start = time.time()

# Module logger
log = logging.getLogger(__name__)

# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WORKFLOW')
rgi_version = '62'
rgi_region = '13'

# Here we override some of the default parameters
# How many grid points around the glacier?
# Make it large if you expect your glaciers to grow large:
# here, 80 is more than enough
cfg.PARAMS['border'] = 80
cfg.PARAMS['use_multiprocessing'] = True

# My configs
cfg.PARAMS['hydro_month_nh'] = 1
climate='W5E5'

# import the MSsandbox modules
from MBsandbox.mbmod_daily_oneflowline import process_w5e5_data, TIModel, BASENAMES, process_era5_daily_data

# Local working directory (where OGGM will write its output)
WORKING_DIR = utils.gettempdir('Experiment')
utils.mkdir(WORKING_DIR, reset=False)
cfg.PATHS['working_dir'] = WORKING_DIR

In [ ]:
# RGI file
path = utils.get_rgi_region_file(rgi_region, version=rgi_version)
rgidf = gpd.read_file(path)
log.workflow('Starting OGGM run')
log.workflow('Number of glaciers: {}'.format(len(rgidf)))
#rgi_df = rgidf.iloc[[0, -1]]
#rgi_df = rgidf

# Go - get the pre-processed glacier directories
#gdirs = workflow.init_glacier_regions(rgidf[12670:12680], from_prepro_level=2)
gdirs = workflow.init_glacier_regions(rgidf, from_prepro_level=2)
rgidf[["RGIId", "CenLon", "CenLat"]].to_csv("rgi_" + rgi_region + "_lon_lat.csv")

In [ ]:
# My Experiment
temps=[]
# Get climate Data
for j,gdir in enumerate(gdirs):
    process_w5e5_data(gdir, climate_type=climate, temporal_resol='monthly', y0 = 2000, y1=2010)
    file = gdir.get_filepath("climate_historical", filesuffix="_monthly_W5E5")
    ds = xr.open_dataset(file)
    w5e5_coords = [ds.ref_pix_lat, ds.ref_pix_lon]
    log.workflow(f'Completed {j} out of {len(rgi_df)}')
# Combine with glacier data
    ds["month"] = ds.time.dt.month
    # Get the topo data and the glacier mask
    with xr.open_dataset(gdir.get_filepath('gridded_data')) as ts:
        topo = ts.topo
    for i in range(1, 13):
        temps.append([gdir.rgi_id, ds.ref_hgt, w5e5_coords, i, topo.data.min(), gdir.rgi_area_km2,  ds.groupby("month")[i].rolling(time=10).mean().dropna("time")["temp"].values[0], ds.groupby("month")[i].rolling(time=10).mean().dropna("time")["gradient"].values[0]])
df = pd.DataFrame (temps, columns = ['rgi_id', 'w5e5_hgt', 'w5e5_coords', 'month', 'terminus_alt', 'area', 'temp', 'gradient' ])
df.temp += df.gradient * (df.terminus_alt - df.w5e5_hgt)
df

In [ ]:
ds

In [ ]:
df = pd.DataFrame (temps, columns = ['rgi_id', 'w5e5_hgt', 'w5e5_coords', 'month', 'terminus_alt', 'area', 'temp', 'gradient' ])
df.temp += df.gradient * (df.terminus_alt - df.w5e5_hgt)
df

In [ ]:
df3 = df.groupby("rgi_id").min()
df3["min_annual_temp"] = df3["temp"]
df3[["weighted_subzero_alt", "weighted_terminus_alt", "subzero_alt"]] = 0
df3 = df3.reset_index()
df3

In [ ]:
for i in range(0,df3.shape[0]):
    df3.loc[i,"month"] = df.loc[df.temp==df3.min_annual_temp[i], "month"].values[0]
    df3.loc[i,"gradient"] = df.loc[df.temp==df3.min_annual_temp[i], "gradient"].values[0]
    df3.loc[i,"terminus_alt"] = df.loc[df.temp==df3.min_annual_temp[i], "terminus_alt"].values[0]
    df3.loc[i, "subzero_alt"] = -df3.loc[i,"min_annual_temp"]/df3.loc[i,"gradient"] + df3.loc[i,"terminus_alt"]
    df3.loc[i, "weighted_subzero_alt"] = df3.loc[i, "area"] * df3.loc[i, "subzero_alt"]
    df3.loc[i, "weighted_terminus_alt"] = df3.loc[i, "area"]* df3.loc[i, "terminus_alt"]
    

In [ ]:
df4 = df3[['rgi_id', 'w5e5_hgt', 'w5e5_coords', 'weighted_subzero_alt', 'weighted_terminus_alt']]
df4

In [ ]:
df4['w5e5_coords'] = df4['w5e5_coords'].astype(str)
df4.dtypes
df5 = df4.groupby("w5e5_hgt").mean().reset_index()
for i in range(0,df5.shape[0]):
    df5.loc[i,"w5e5_coords"] = df4.loc[df4.w5e5_hgt==df5.w5e5_hgt[i], "w5e5_coords"].values[0]

df5["alt_diff"] = df5["weighted_terminus_alt"] - df5["weighted_subzero_alt"]
df6 = df5[['w5e5_coords', 'alt_diff']]
df6

In [ ]:
df6['w5e5_coords']=df6['w5e5_coords'].str.strip('[]').str.split(',')
df6['lat'], df6['lon'] = zip(*df6.pop('w5e5_coords'))
df6 = df6[["lat", "lon", "alt_diff"]]
df6

In [ ]:
ds2 = xr.open_dataset("/home/bsurya/work/w5e5_orog/Global_RiverSlope_HydroSTN30_06min_Static.nc", decode_times=False)
for i in range(0,df6.shape[0]):
    df6.loc[i,"slope"] = ds2.sel(latitude=df6.lat[i], longitude=df6.lon[i], method="nearest").RiverSlope.values[0]
df6

In [ ]:
mask1=  (df6.alt_diff > 10)
mask2 = (df6.slope > 0.01)
df6[mask2 & mask1].count()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df6.index, df6.alt_diff, color="black", label = "Condition 1 failure")
plt.scatter(df6[mask1].index, df6[mask1].alt_diff, color="blue", label = "Condition 2 failure")
plt.scatter(df6[mask2].index, df6[mask2].alt_diff, color="red", label = "All conditions pass")
plt.ylabel("Terminus and Subzero alt difference")
plt.xlabel("W5E5 grid cells")
plt.legend()